In [ ]:
# Import required libraries
import pandas as pd
import matplotlib.pyplot as plt
from QuantConnect import *
from QuantConnect.Indicators import *
import config as c

# Initialize QuantBook
qb = QuantBook()

# Define symbol based on your config
symbol = qb.AddEquity(c.symbol_filter_parameter_static_universe[0]).Symbol

# Fetch historical data
history = qb.History(symbol, 360, c.finest_resolution).loc[symbol]

# Initialize indicators with parameters from config
ema_short = ExponentialMovingAverage(c.buy_parameter_ema_short_periods)
ema_long = ExponentialMovingAverage(c.buy_parameter_ema_long_periods)
macd = MovingAverageConvergenceDivergence(12, 26, 9)
rsi = RelativeStrengthIndex(c.buy_parameter_rsi_periods)
sto = Stochastic(c.buy_parameter_stochastic_rsi_periods, 3, 3)

# Create a DataFrame to store indicator values
indicator_df = pd.DataFrame(index=history.index, columns=['EMA Short', 'EMA Long', 'MACD', 'MACD Signal', 'MACD Histogram', 'RSI', 'Stochastic'])

# Calculate indicators for each historical data row
for time, row in history.iterrows():
    # Update indicators with current row data
    ema_short.Update(time, row.close)
    ema_long.Update(time, row.close)
    macd.Update(time, row.close)
    rsi.Update(time, row.close)
    sto.Update(time, row.high, row.low, row.close)
    
    # Store indicator values in DataFrame
    indicator_df.at[time, 'EMA Short'] = ema_short.Current.Value if ema_short.IsReady else None
    indicator_df.at[time, 'EMA Long'] = ema_long.Current.Value if ema_long.IsReady else None
    indicator_df.at[time, 'MACD'] = macd.Current.Value if macd.IsReady else None
    indicator_df.at[time, 'MACD Signal'] = macd.Signal.Current.Value if macd.IsReady else None
    indicator_df.at[time, 'MACD Histogram'] = macd.Current.Value - macd.Signal.Current.Value if macd.IsReady else None
    indicator_df.at[time, 'RSI'] = rsi.Current.Value if rsi.IsReady else None
    indicator_df.at[time, 'Stochastic'] = sto.Current.Value if sto.IsReady else None

# Plotting
plt.figure(figsize=(15, 10))

# EMA plot
plt.subplot(3, 1, 1)
plt.title(f"EMAs for {symbol}")
plt.plot(indicator_df['EMA Short'], label="EMA Short")
plt.plot(indicator_df['EMA Long'], label="EMA Long")
plt.legend()

# MACD plot
plt.subplot(3, 1, 2)
plt.title(f"MACD for {symbol}")
plt.plot(indicator_df['MACD'], label="MACD")
plt.plot(indicator_df['MACD Signal'], label="MACD Signal")
plt.bar(indicator_df.index, indicator_df['MACD Histogram'], label='MACD Histogram', color='grey')
plt.legend()

# RSI and Stochastic plot
plt.subplot(3, 1, 3)
plt.title(f"RSI & Stochastic for {symbol}")
plt.plot(indicator_df['RSI'], label="RSI")
plt.plot(indicator_df['Stochastic'], label="Stochastic")
plt.legend()

plt.tight_layout()
plt.show()
